In [ ]:
import pandas as pd
import os
import re
import openpyxl
from typing import List, Any, Callable, Set, Tuple
from decompa import DECompa # The de-compounding library not able to download (Bosch Firewall)

In [11]:
SOURCE_FILENAME = 'TEST_Data_for_REGEX.xlsx'
SOURCE_COLUMN_NAME = 'Typkurzbezeichnung'
CONDITION_COLUMN = 'MArt'
TARGET_FLAG_COLUMN = 'B' # Will contain 1 or ""
TARGET_REASON_COLUMN = 'lsg'    # Will contain the detailed reason

# Conditional Values
CONDITION_VALUES = ['BREX', 'KAUF'] # Check if MArt is one of these

# Resource File Names
STOPWORDS_FILENAME = 'german_stopwords.txt'
DICTIONARY_FILENAME = 'german_dictionary.txt'

# Validation Parameters
MIN_TOKEN_LENGTH = 3

In [ ]:

# ==============================================================================
# 1. SETUP AND RESOURCE LOADING FUNCTIONS
# ==============================================================================

def load_text_file_to_set(filename: str) -> Set[str]:
    """Loads a .txt file (one item per line) into a lowercase set for fast lookups."""
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            # Use a set for O(1) average time complexity checks (very fast)
            # .strip() removes whitespace, .lower() ensures case-insensitivity
            return {line.strip().lower() for line in f if line.strip()}
    except FileNotFoundError:
        print(f"⚠️ WARNING: Resource file '{filename}' not found. This check will be skipped.")
        return set()

def load_data(file_path: str) -> pd.DataFrame:
    """Loads data from an Excel file into a pandas DataFrame."""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file was not found at: {file_path}")
    print(f"Loading data from '{os.path.basename(file_path)}'...")
    return pd.read_excel(file_path)

def save_data_to_excel(file_path: str, results_df: pd.DataFrame):
    """
    Opens an Excel workbook and updates multiple columns with new data,
    preserving all existing formatting.
    """
    print(f"Opening workbook to write results...")
    try:
        workbook = openpyxl.load_workbook(file_path)
        sheet = workbook.active
    except FileNotFoundError:
        raise FileNotFoundError(f"Failed to open workbook. Not found at: {file_path}")

    # Create a mapping from column name to its numerical index
    header = {cell.value: cell.column for cell in sheet[1]}
    
    # Update each column specified in the results DataFrame
    for col_name in results_df.columns:
        if col_name not in header:
            print(f"⚠️ WARNING: Target column '{col_name}' not found in Excel. It will be skipped.")
            continue
        
        target_col_num = header[col_name]
        print(f"Writing data to column '{col_name}'...")
        for index, value in results_df[col_name].items():
            sheet.cell(row=index + 2, column=target_col_num, value=value)
            
    try:
        workbook.save(file_path)
    except PermissionError:
        raise PermissionError(f"Could not save. Is the file '{os.path.basename(file_path)}' open in Excel?")

In [13]:

class TextValidator:
    """A class to encapsulate the entire validation logic and its resources."""
    
    def __init__(self, stopwords_path: str, dictionary_path: str):
        print("Initializing validator and loading resources...")
        # Load resources into memory once
        self.stopwords = load_text_file_to_set(stopwords_path)
        self.dictionary = load_text_file_to_set(dictionary_path)
        self.decompounder = DECompa()

    def _stage1_invalid_char_check(self, text: str) -> Tuple[bool, str]:
        """Stage 1: Checks for forbidden characters."""
        pattern = r"[^A-Z0-9*.,\"%/ :;=<>()–-&Ä´`_Ø+ \r\n-]"
        if re.search(pattern, text, re.IGNORECASE):
            return True, "Contains Invalid Characters"
        return False, ""

    def _stage2_german_language_analysis(self, text: str) -> Tuple[bool, str]:
        """Stage 2: Tokenizes text and runs checks on each token."""
        # Step A: Extract tokens of 3 or more letters
        tokens = re.findall(r'[A-Za-z]{' + str(MIN_TOKEN_LENGTH) + r',}', text)
        
        for token in tokens:
            token_lower = token.lower()
            
            # Check 1: Stop words
            if token_lower in self.stopwords:
                return True, f"Contains Stop Word: {token}"
            
            # Check 2: Direct dictionary match
            if token_lower in self.dictionary:
                return True, f"Contains Dictionary Word: {token}"
                
            # Check 3: Morphological check (only if not a direct match)
            parts = self.decompounder.split(token)
            # The word was successfully de-compounded if it has more than 1 part
            # and all of its parts are known dictionary words.
            if len(parts) > 1 and all(part.lower() in self.dictionary for part in parts):
                return True, f"Contains Compound Word: {token}"
        
        return False, ""

    def validate(self, text: Any) -> Tuple[int, str]:
        """
        Runs the full validation pipeline on a single piece of text.
        Returns a tuple: (flag, reason). e.g., (1, "Contains Stop Word: FUER")
        """
        if not isinstance(text, str) or not text.strip():
            return "", "" # Not an error, just empty

        # Run Stage 1
        is_error, reason = self._stage1_invalid_char_check(text)
        if is_error:
            return 1, reason

        # Run Stage 2
        is_error, reason = self._stage2_german_language_analysis(text)
        if is_error:
            return 1, reason

        # If all checks pass
        return "", ""

In [14]:

def main():
    """Main function to orchestrate the entire data processing workflow."""
    process_name = "Advanced Material Master Data Validation"
    print(f"--- Starting: {process_name} ---")
    try:
        # Get absolute paths for resource files
        script_dir = os.path.dirname(__file__)
        excel_file_path = os.path.join(script_dir, SOURCE_FILENAME)
        stopwords_path = os.path.join(script_dir, STOPWORDS_FILENAME)
        dictionary_path = os.path.join(script_dir, DICTIONARY_FILENAME)
        
        # Step 1: Load data and initialize validator
        df = load_data(excel_file_path)
        validator = TextValidator(stopwords_path, dictionary_path)
        
        # Check that all necessary columns exist
        required_cols = [SOURCE_COLUMN_NAME, CONDITION_COLUMN]
        if not all(col in df.columns for col in required_cols):
            raise ValueError(f"Required columns ({required_cols}) not found.")

        # Step 2: Apply the validation logic row-by-row
        print("Running conditional validation pipeline...")
        
        def apply_full_validation(row: pd.Series) -> pd.Series:
            """Applies validation logic to a single row and returns results for both target columns."""
            flag, reason = "", "" # Default values are "no error"
            
            # Conditional check: only run validation if MArt is one of the specified values
            if row[CONDITION_COLUMN] in CONDITION_VALUES:
                source_text = row[SOURCE_COLUMN_NAME]
                flag, reason = validator.validate(source_text)
            
            return pd.Series([flag, reason], index=[TARGET_FLAG_COLUMN, TARGET_REASON_COLUMN])

        # df.apply returns a new DataFrame with our two new columns
        results_df = df.apply(apply_full_validation, axis=1)

        # Step 3: Save the results back to the original Excel file
        save_data_to_excel(excel_file_path, results_df)

        print("\n" + "="*50)
        print("✅ SUCCESS: Workflow completed without errors.")
        print("="*50)

    except (FileNotFoundError, ValueError, PermissionError) as e:
        print(f"\n❌ ERROR: The script failed. Reason: {e}")
    except Exception as e:
        print(f"\n❌ An unexpected error occurred: {e}")

# ==============================================================================
# 4. SCRIPT ENTRY POINT
# ==============================================================================

if __name__ == "__main__":
    main()

--- Starting: Advanced Material Master Data Validation ---

❌ An unexpected error occurred: name '__file__' is not defined


In [ ]:
german_words = load_text_file_to_set("german_words.txt")
french_words = load_text_file_to_set("")
stop_words = load_text_file_to_set("german_stopwords.txt")
english_words = load_text_file_to_set("english_words.txt")

Test = load_text_file_to_set("Typkurzbezeichnung TEST V1.txt")

def stage2_word_analysis(text: str) -> Tuple[bool, str]:
        """Stage 2: Tokenizes text and runs checks on each token."""

        print(text)
        # Step A: Extract tokens of 3 or more letters
        tokens = re.findall(r'[A-Za-z]{' + str(MIN_TOKEN_LENGTH) + r',}', text)
        
        for token in tokens:
            token_lower = token.lower()
            
            # Check 1: Stop words
            if token_lower in stop_words:
                return True, f"Contains Stop Word: {token}"
            
            #Check 2: English Words
            if token_lower in english_words:
                 continue
            
            # Check 2: Direct dictionary match
            if token_lower in german_words:
                return True, f"Contains Dictionary Word: {token}"
            
            #check 3: Compound word check 
            for word in german_words:
                 if len(word)>=4 and word in token_lower:
                      return True, f"Contains compound partial word: {word} for Token: {token}"
        
        return False, ""

for typcode in Test: 
    print(stage2_word_analysis(typcode))



0500x0500x0055/0384 s1--a teil1  d900398
(True, 'Contains Stop Word: teil')
druckstueck r902774821
(True, 'Contains compound partial word: rucks for Token: druckstueck ')
grund,wsm
(True, 'Contains Dictionary Word: grund')
vorstufe unlackiert fuer r917016850
(True, 'Contains Stop Word: vorstufe')
agev2-26970-aa-101 filteranbaublock
(True, 'Contains compound partial word: block for Token: filteranbaublock ')
20-20 vhm i-smart schrupp
(True, 'Contains compound partial word: rupp for Token: schrupp ')
cz112842 for 3032376-400
(True, 'Contains Stop Word: for')
vorstufe unlackiert fuer r917016819
(True, 'Contains Stop Word: vorstufe')
vorstufe unlackiert fuer r917016870
(True, 'Contains Stop Word: vorstufe')
stempel 75.0.0955 stl.
(True, 'Contains Stop Word: stempel')
vorstufe unlackiert fuer r917016811
(True, 'Contains Stop Word: vorstufe')
druckstueck r902775048
(True, 'Contains compound partial word: rucks for Token: druckstueck ')
speicher,2-fach
(True, 'Contains Stop Word: speicher')
a